# Análise Final — E-commerce

Este notebook apresenta análises de negócio com foco em faturamento, ticket médio,
comportamento temporal, geográfico e operacional dos pedidos.

In [1]:
# Importando Bibliotecas
import pandas as pd
import plotly.express as px

In [2]:
# Leitura e preparação dos dados
df = pd.read_csv('ecommerce.csv')
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d')

In [3]:
# Faturamento por categoria
faturamento_categoria = (
    df.groupby('product_category')['total_value']
      .sum()
      .reset_index()
      .sort_values(by='total_value', ascending=False)
)

fig = px.bar(
    x=faturamento_categoria['product_category'],
    y=faturamento_categoria['total_value'],
    title='Faturamento por Categoria'
)

fig.update_xaxes(title='Categoria', tickangle=-45)
fig.update_yaxes(title='Faturamento (R$)')
fig.show()

In [4]:
# Ticket médio por categoria (Número de pedidos únicos por categoria)
n_pedidos = (
    df.groupby('product_category')['order_id']
      .nunique()
      .reset_index()
)

n_pedidos.columns = ['categoria', 'total_pedidos_unicos']

# Cálculo do Ticket Médio
ticket_medio_categoria = pd.merge(
    n_pedidos,
    faturamento_categoria,
    left_on='categoria',
    right_on='product_category',
    how='inner'
)

ticket_medio_categoria['ticket_medio'] = (
    ticket_medio_categoria['total_value'] /
    ticket_medio_categoria['total_pedidos_unicos']
)

ticket_medio_categoria = ticket_medio_categoria.sort_values(
    by='ticket_medio', ascending=False
)

In [5]:
# Gráfico de ticket médio
fig = px.bar(
    x=ticket_medio_categoria['categoria'],
    y=ticket_medio_categoria['ticket_medio'],
    title='Ticket Médio por Categoria'
)

fig.update_xaxes(title='Categoria', tickangle=45)
fig.update_yaxes(title='Ticket Médio (R$)')
fig.show()

In [6]:
# Evolução mensal do faturamento
faturamento_mes_ano = (
    df.groupby(df['order_date'].dt.to_period('M'))['total_value']
      .sum()
      .reset_index()
)

faturamento_mes_ano['order_date'] = faturamento_mes_ano['order_date'].astype(str)

fig = px.line(
    faturamento_mes_ano,
    x='order_date',
    y='total_value',
    title='Evolução Mensal do Faturamento (R$)',
    markers=True
)

fig.update_xaxes(tickangle=-45)
fig.show()

In [7]:
# Faturamento por estado
faturamento_estado_cliente = (
    df.groupby('customer_state')['total_value']
      .sum()
      .reset_index()
      .sort_values(by='total_value', ascending=False)
)

fig = px.bar(
    x=faturamento_estado_cliente['customer_state'],
    y=faturamento_estado_cliente['total_value'],
    title='Faturamento por Estado'
)

fig.update_xaxes(title='Estado', tickangle=-45)
fig.update_yaxes(title='Faturamento (R$)')
fig.show()

In [8]:
# Status dos pedidos
status_counts = df['order_status'].value_counts().reset_index()
status_counts.columns = ['status', 'quantidade']

fig = px.pie(
    status_counts,
    names='status',
    values='quantidade',
    title='Distribuição dos Status dos Pedidos'
)

fig.show()

In [9]:
# Faturamento por tipo de pagamento
tipo_canal = df.groupby('payment_type')['total_value'].sum().reset_index()

fig = px.bar(
    x=tipo_canal['payment_type'],
    y=tipo_canal['total_value'],
    title='Faturamento por Tipo de Pagamento'
)

fig.update_xaxes(title='Tipo de Pagamento')
fig.update_yaxes(title='Faturamento (R$)')
fig.show()

## Principais Insights

- Algumas categorias concentram maior faturamento, mas não necessariamente maior ticket médio.
- O ticket médio varia significativamente entre categorias, indicando perfis distintos de consumo.
- Existe uma tendência de crescimento/variação mensal no faturamento.
- Determinados estados concentram a maior parte da receita.
- A maioria dos pedidos encontra-se com status entregue.